In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.pipeline import Pipeline, FeatureUnion

import functions as f
import classes as c

In [2]:
train_df = pd.read_csv("datasets/train.csv").set_index("Id")
train_df[pd.isnull(train_df)] = np.NaN

X = train_df.drop("SalePrice", axis=1)
y = train_df["SalePrice"].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [3]:
drop_col = []
nominal_col = ["MSSubClass", "MSZoning", "LandContour", "LotConfig",
               "Neighborhood", "Condition1", "Condition2", "BldgType", "HouseStyle",
               "RoofStyle", "RoofMatl", "Exterior1st", "Exterior2nd", "MasVnrType",
               "Foundation", "Heating", "Electrical", "GarageType", "Fence", "MiscFeature", 
              "SaleType", "SaleCondition"]
nominal_col_cat = f.unique_values(train_df[nominal_col])
binary_col_cat = f.unique_values(train_df[binary_col])

binary_col = ["Street", "CentralAir"]
ordinal_col = ["Alley", "LotShape", "Utilities", "LandSlope", "OverallQual", "OverallCond",
               "ExterQual", "ExterCond", "BsmtQual", "BsmtCond", "BsmtExposure", "BsmtFinType1",
               "BsmtFinType2", "HeatingQC", "KitchenQual", "Functional", "FireplaceQu",
               "GarageFinish", "GarageQual", "GarageCond", "PavedDrive", "PoolQC"]
count_col = ["BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "KitchenAbvGr",
                "TotRmsAbvGrd", "Fireplaces", "GarageCars"]
interval_col = ["LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF",
                "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea",
                "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch",
                "ScreenPorch", "PoolArea", "MiscVal"]
time_col = ["YearBuilt", "YearRemodAdd", "GarageYrBlt", "MoSold", "YrSold"]
miss_vall_col = ["LotFrontage", "MasVnrArea", "MasVnrType", "Electrical"]

In [10]:
f.unique_values(train_df[binary_col])

[array(['Pave', 'Grvl'], dtype=object), array(['Y', 'N'], dtype=object)]

In [4]:
f.lists_eq(train_df.columns, nominal_col+binary_col+ordinal_col+count_col+interval_col+time_col+["SalePrice"])

True

In [5]:
count_interval_time_pip = Pipeline([
                ("selector", c.DataFrameSelector(columns=interval_col+count_col+time_col)),
                ("standard", StandardScaler())
            ])
nominal_pip = Pipeline([
                ("selector", c.DataFrameSelector(columns=nominal_col)),
                ("imputer", c.DataFrameImputer(strategy="constant", columns=nominal_col)),
                ("one_hot_encoder", OneHotEncoder(sparse=False))
            ])
binary_ordinal_pip = Pipeline([
                ("selector", c.DataFrameSelector(columns=binary_col+ordinal_col)),
                ("imputer", c.DataFrameImputer(strategy="constant", columns=binary_col+ordinal_col)),
                ("one_hot_encoder", OneHotEncoder(sparse=False))
            ])
missing_vallue_pip = Pipeline([
                ("selector", c.DataFrameSelector(columns=X_train.columns)),
                ("mas_vnr_imputer", c.MasVnrImputer()),
                ("mean_imputer", c.DataFrameImputer(strategy="mean", columns=["LotFrontage"])),
                ("most_frequent_imputer", c.DataFrameImputer(strategy="most_frequent", columns=["Electrical"]))
            ])


In [6]:
full_pip = Pipeline([
    ("missing_vallue", missing_vallue_pip),
    ("feature_union", FeatureUnion([
        ("count_interval_time", count_interval_time_pip),
        ("nominal", nominal_pip),
        ("binary_ordinal_pip", binary_ordinal_pip)]))
])

In [8]:
X_train_tr = full_pip.fit_transform(X_train)
X_test_tr = full_pip.transform(X_test)

C:\Users\Paul\AppData\Local\conda\conda\envs\handson-ml\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Paul\AppData\Local\conda\conda\envs\handson-ml\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Users\Paul\AppData\Local\conda\conda\envs\handson-ml\lib\site-packages\sklearn\pipeline.py:451: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


ValueError: Found unknown categories ['RRNn', 'PosA'] in column 6 during transform

In [ ]:
count_interval_time_pip.fit_transform(train_df)